# practice 1 : probe


probeinterface is a tool to handle blablabla...



In [12]:
#%matplotlib inline
%matplotlib widget

In [13]:
import probeinterface as pi
from probeinterface.plotting import plot_probe


## Some probe are already implemented (cambridge neurotec)

In [18]:
manufacturer = 'cambridgeneurotech'
probe_name = 'ASSY-236-H6'

probe = pi.get_probe(manufacturer, probe_name)
print(probe)

cambridgeneurotech - ASSY-236-H6 - 64ch


In [20]:
plot_probe(probe, with_contact_id=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<matplotlib.collections.PolyCollection at 0x7fb00d22ee50>,
 <matplotlib.collections.PolyCollection at 0x7fb00d1df8b0>)

## Step 1 : construct a probe from channel position

todo:
generate file csv with positions
get images from neuronexus


## Step 2 : wire to device channel (aka pathway)

## Step 3 : save the "probe wired" into json